# Running a CAT based on a synthetic correct/incorrect 3PL IRT model

This example shows how to run a CAT based on a synthetic correct/incorrect 3PL
IRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: AbilityVarianceStateCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
using ComputerAdaptiveTesting.Responses: BooleanResponse
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
using ComputerAdaptiveTesting.DummyData: dummy_full, std_normal, SimpleItemBankSpec, StdModel3PL
using ComputerAdaptiveTesting.MathTraits
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel3PL(), OneDimContinuousDomain(), BooleanResponse());
    num_questions=100,
    num_testees=3
)

(ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}}([0.0, 0.4347592962774931, 0.0, 0.2767106342986333, 0.0, 0.0, 0.13003682099947314, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.1350686659369462, 0.22854353128614596, 0.4196031218377332, 0.28394485427949445, 0.0, 0.19397233201037478, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [-1.1511815194158441, -1.8930417835431035, 0.7218448817498232, -2.131325054334863, -0.41541681793444635, -0.5632074113382671, -0.8420369246967712, 0.6675476968377481, 1.1575706468042362, 0.28518873019760305  …  -2.5665357851525155, -1.4925011873105412, -1.4025405017842998, 1.1197502505405268, 0.5400698890495721, 1.897167577491841, -0.07256688641276972, -1.2816710230095782, 1.053267153908723, -0.2466351471297366], [1.336691743586448, 1.1713972116329234, 1.2467056702282402, 1.319683612410

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [3]:
max_questions = 99
integrator = FixedGKIntegrator(-6, 6, 80)
dist_ability_est = PriorAbilityEstimator(std_normal)
ability_estimator = MeanAbilityEstimator(dist_ability_est, integrator)
rules = CatRules(
    ability_estimator,
    AbilityVarianceStateCriterion(dist_ability_est, integrator),
    FixedItemsTerminationCondition(max_questions)
)

points = 500
xs = range(-2.5, 2.5, length=points)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator)
for testee_idx in axes(responses, 2)
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[testee_idx]
    abs_err = abs(θ - true_θ)
end

Make a plot showing how the estimated value evolves during the CAT.
We also plot the 'true' values used to generate the responses.

In [4]:
conv_lines_fig = ability_evolution_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

Make an interactive plot, showing how the distribution of the ability
likelihood evolves.

In [5]:
conv_dist_fig = lh_evolution_interactive(recorder; abilities=abilities)
conv_dist_fig

Figure()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*